In [4]:
"""
===============================================================================
이더리움 트레이딩 백테스팅 시스템 (전체 전략 구현)
===============================================================================
"""

import pandas as pd
import numpy as np
import os
from scipy import stats
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# ========================================================================
# 1. 경로 및 전역 설정
# ========================================================================

timestamp = '2025-10-26'
RESULT_DIR = os.path.join("../model_results", timestamp)
fold_results_dir = os.path.join(RESULT_DIR, "fold_results", "direction")
output_dir = os.path.join(RESULT_DIR, "backtest_results")

# 전체 15개 전략 정의
STRATEGY_INFO = {
    'kelly': 'Kelly Criterion (1956)', 
    'aggressive': 'Fixed 30%', 
    'neutral': 'Confidence 15%',
    'conservative': 'Fixed 5%', 
    'ultra_aggressive': 'Fixed 80%', 
    'fixed_50': 'Fixed 50%',
    'fixed_01': 'Fixed 1%', 
    'volatility_scaled': 'Inverse Vol (cap 30%)',
    'inverse_volatility': 'Inverse Vol (cap 50%)', 
    'trend_following': 'Momentum Binary',
    'momentum': 'Momentum Scaled', 
    'reinforcement_learning': 'Q-learning Adaptive',
    'regime_adaptive_balanced': 'Regime Balanced', 
    'regime_adaptive_defensive': 'Regime Defensive',
    'regime_adaptive_aggressive': 'Regime Aggressive'
}

# ========================================================================
# 2. 데이터 검증 함수
# ========================================================================

def validate_dataframe(df, model_name):
    """예측 결과 DataFrame의 유효성 검증"""
    required_cols = ['date', 'pred_direction', 'actual_return', 'correct', 'confidence', 'max_proba']
    missing = [col for col in required_cols if col not in df.columns]
    
    if missing or len(df) < 10 or df['actual_return'].isna().all():
        return False
    return True

# ========================================================================
# 3. 강화학습 에이전트 (Q-Learning)
# ========================================================================

class RLAgent:
    """Q-learning 기반 포지션 사이징 에이전트"""
    
    def __init__(self):
        self.q_table = {}
        self.actions = [0.01, 0.05, 0.1, 0.15, 0.2, 0.3, 0.5]
        self.lr = 0.15
        self.epsilon = 0.3
        self.gamma = 0.95
        
    def get_state(self, volatility, trend, streak, drawdown):
        """시장 상태를 이산화하여 상태 튜플 생성"""
        vol = 'high' if volatility > 0.025 else 'med' if volatility > 0.015 else 'low'
        tr = 'bull' if trend > 0.01 else 'bear' if trend < -0.01 else 'flat'
        st = 'hot' if streak >= 3 else 'cold' if streak <= -3 else 'neutral'
        dd = 'severe' if drawdown < -0.15 else 'moderate' if drawdown < -0.08 else 'safe'
        return (vol, tr, st, dd)
    
    def choose_action(self, state):
        """ε-greedy 정책으로 행동 선택"""
        if state not in self.q_table:
            self.q_table[state] = {a: 0.0 for a in self.actions}
        
        if np.random.random() < self.epsilon:
            return np.random.choice(self.actions)
        
        q_values = self.q_table[state]
        if all(v == 0.0 for v in q_values.values()):
            return 0.1
        return max(q_values, key=q_values.get)
    
    def update(self, state, action, reward, next_state):
        """Q-value 업데이트"""
        if state not in self.q_table:
            self.q_table[state] = {a: 0.0 for a in self.actions}
        if next_state not in self.q_table:
            self.q_table[next_state] = {a: 0.0 for a in self.actions}
        
        closest = min(self.actions, key=lambda x: abs(x - action))
        current_q = self.q_table[state][closest]
        max_next_q = max(self.q_table[next_state].values())
        self.q_table[state][closest] = current_q + self.lr * (reward + self.gamma * max_next_q - current_q)
        
    def decay_epsilon(self):
        """탐험 확률을 점진적으로 감소"""
        self.epsilon = max(0.05, self.epsilon * 0.995)

# ========================================================================
# 4. 백테스팅 엔진 (전체 15개 전략 구현)
# ========================================================================

class Backtester:
    """트레이딩 백테스팅 시뮬레이터"""
    
    def __init__(self, initial_capital=10000000, fee=0.0005, slippage=0.0015, min_order=5000):
        self.initial_capital = float(initial_capital)
        self.fee = float(fee)
        self.slippage = float(slippage)
        self.min_order = min_order
        self.min_trades = 30

    def detect_regime(self, returns, lookback=30):
        """시장 체제(regime) 감지"""
        if len(returns) < lookback:
            return "sideways", 0.15
        
        recent = returns[-lookback:]
        cum_ret = np.sum(recent)
        volatility = np.std(recent)
        
        if cum_ret > 0.10 and volatility > 0.025:
            return "bull_strong", 0.40
        elif cum_ret > 0.05 and volatility < 0.020:
            return "bull_steady", 0.30
        elif cum_ret < -0.10 and volatility > 0.025:
            return "bear_crash", 0.05
        elif cum_ret < -0.05:
            return "bear_volatile", 0.08
        else:
            return "sideways", 0.15

    def kelly_fraction(self, profits):
        """Kelly Criterion 계산"""
        if len(profits) < 10:
            return 0.05
        
        profits = np.array(profits)
        wins = profits[profits > 0]
        losses = profits[profits < 0]
        
        if len(wins) == 0 or len(losses) == 0 or np.mean(losses) == 0:
            return 0.05
        
        win_rate = len(wins) / len(profits)
        avg_win = np.mean(wins)
        avg_loss = np.abs(np.mean(losses))
        win_loss_ratio = avg_win / avg_loss
        kelly = (win_rate * win_loss_ratio - (1 - win_rate)) / win_loss_ratio
        
        return np.clip(kelly * 0.5, 0.01, 0.2)

    def calculate_position(self, df, i, strategy, recent_profits, recent_returns, 
                          streak, peak, capital, agent=None):

        confidence = df.iloc[i]['confidence']
        max_proba = df.iloc[i]['max_proba']
        
        # === 전략 1: Kelly Criterion ===
        if strategy == 'kelly':
            if len(recent_profits) >= 20:
                return self.kelly_fraction(recent_profits[-50:])
            return 0.05
        
        # === 전략 2: Fixed 30% (공격적) ===
        elif strategy == 'aggressive':
            return 0.3
        
        # === 전략 3: Confidence 기반 15% ===
        elif strategy == 'neutral':
            return np.clip(max_proba * 0.15, 0.01, 0.15)
        
        # === 전략 4: Fixed 5% (보수적) ===
        elif strategy == 'conservative':
            return 0.05
        
        # === 전략 5: Fixed 80% (초공격적) ===
        elif strategy == 'ultra_aggressive':
            return 0.8
        
        # === 전략 6: Fixed 50% ===
        elif strategy == 'fixed_50':
            return 0.5
        
        # === 전략 7: Fixed 1% (초보수적) ===
        elif strategy == 'fixed_01':
            return 0.01
        
        # === 전략 8: 변동성 역수 (cap 30%) ===
        elif strategy == 'volatility_scaled':
            if len(recent_returns) >= 20:
                vol = np.std(recent_returns[-20:])
                vol = max(vol, 0.01)
                return np.clip(0.1 / vol, 0.01, 0.3)
            return 0.1
        
        # === 전략 9: 변동성 역수 (cap 50%) ===
        elif strategy == 'inverse_volatility':
            if len(recent_returns) >= 20:
                vol = np.std(recent_returns[-20:])
                vol = max(vol, 0.01)
                return np.clip(0.2 / vol, 0.01, 0.5)
            return 0.15
        
        # === 전략 10: 추세 추종 (Binary) ===
        elif strategy == 'trend_following':
            if len(recent_returns) >= 10:
                trend = np.sum(recent_returns[-10:])
                return 0.3 if trend > 0 else 0.05
            return 0.1
        
        # === 전략 11: 모멘텀 Scaled ===
        elif strategy == 'momentum':
            if len(recent_returns) >= 10:
                momentum = np.sum(recent_returns[-10:])
                return np.clip(0.1 + 0.2 * (momentum / 0.1), 0.01, 0.3)
            return 0.1
        
        # === 전략 12: 강화학습 (Q-learning) ===
        elif strategy == 'reinforcement_learning':
            if agent is None:
                return 0.1
            
            volatility = np.std(recent_returns[-20:]) if len(recent_returns) >= 20 else 0.02
            trend = np.mean(recent_returns[-10:]) if len(recent_returns) >= 10 else 0
            drawdown = (capital - peak) / peak if peak > 0 else 0
            state = agent.get_state(volatility, trend, streak, drawdown)
            
            return agent.choose_action(state)
        
        # === 전략 13: Regime Adaptive Balanced ===
        elif strategy == 'regime_adaptive_balanced':
            regime, base_pos = self.detect_regime(recent_returns, 30)
            
            multipliers = {
                "bear_crash": 0.8, "bear_volatile": 0.9, "sideways": 1.0, 
                "bull_steady": 1.3, "bull_strong": 1.5
            }
            
            mult = multipliers.get(regime, 1.0)
            conf_mult = 0.8 + (confidence * 0.8)
            final_pos = base_pos * mult * conf_mult
            
            if regime in ["bear_crash", "bear_volatile"]:
                return np.clip(final_pos, 0.01, 0.15)
            else:
                return np.clip(final_pos, 0.01, 0.7)
        
        # === 전략 14: Regime Adaptive Defensive ===
        elif strategy == 'regime_adaptive_defensive':
            regime, base_pos = self.detect_regime(recent_returns, 30)
            
            multipliers = {
                "bear_crash": 0.6, "bear_volatile": 0.7, "sideways": 0.8, 
                "bull_steady": 1.0, "bull_strong": 1.1
            }
            
            mult = multipliers.get(regime, 1.0)
            conf_mult = 0.8 + (confidence * 0.8)
            final_pos = base_pos * mult * conf_mult
            
            if regime in ["bear_crash", "bear_volatile"]:
                return np.clip(final_pos, 0.01, 0.15)
            else:
                return np.clip(final_pos, 0.01, 0.7)
        
        # === 전략 15: Regime Adaptive Aggressive ===
        elif strategy == 'regime_adaptive_aggressive':
            regime, base_pos = self.detect_regime(recent_returns, 30)
            
            multipliers = {
                "bear_crash": 1.0, "bear_volatile": 1.2, "sideways": 1.5, 
                "bull_steady": 1.8, "bull_strong": 2.0
            }
            
            mult = multipliers.get(regime, 1.0)
            conf_mult = 0.8 + (confidence * 0.8)
            final_pos = base_pos * mult * conf_mult
            
            if regime in ["bear_crash", "bear_volatile"]:
                return np.clip(final_pos, 0.01, 0.15)
            else:
                return np.clip(final_pos, 0.01, 0.7)
        
        # 기본값
        return 0.1

    def backtest(self, df, strategy, confidence_threshold=0.1):
        """백테스팅 실행"""
        df = df.copy().sort_values('date').reset_index(drop=True)
        n = len(df)
        
        agent = RLAgent() if strategy in ['reinforcement_learning'] else None
        
        capital = self.initial_capital
        equity_curve = np.zeros(n)
        trades = []
        recent_profits = []
        recent_returns = []
        streak = 0
        peak = capital
        
        for i in range(n):
            if capital < self.min_order or df.iloc[i]['confidence'] < confidence_threshold:
                equity_curve[i] = capital
                continue
            
            position_fraction = self.calculate_position(
                df, i, strategy, recent_profits, recent_returns, 
                streak, peak, capital, agent
            )
            position_size = capital * position_fraction
            
            if position_size < self.min_order:
                equity_curve[i] = capital
                continue
            
            volatility = np.std(recent_returns[-20:]) if len(recent_returns) >= 20 else 0.02
            dynamic_slippage = self.slippage * (2 if volatility > 0.03 else 1.5 if volatility > 0.02 else 1.0)
            cost = position_size * ((self.fee + dynamic_slippage) * 2)
            
            direction = int(df.iloc[i]['pred_direction'])
            ret = float(df.iloc[i]['actual_return'])
            
            gross = position_size * (ret if direction == 1 else -ret)
            net = gross - cost
            
            prev_capital = capital
            capital = max(0, capital + net)
            peak = max(peak, capital)
            equity_curve[i] = capital
            
            if agent:
                reward = net / position_size if position_size > 0 else 0
                
                next_vol = np.std((recent_returns + [ret])[-20:]) if len(recent_returns) >= 19 else 0.02
                next_trend = np.mean((recent_returns + [ret])[-10:]) if len(recent_returns) >= 9 else 0
                next_dd = (capital - peak) / peak if peak > 0 else 0
                next_state = agent.get_state(next_vol, next_trend, streak, next_dd)
                
                curr_vol = np.std(recent_returns[-20:]) if len(recent_returns) >= 20 else 0.02
                curr_trend = np.mean(recent_returns[-10:]) if len(recent_returns) >= 10 else 0
                curr_dd = (prev_capital - peak) / peak if peak > 0 else 0
                curr_state = agent.get_state(curr_vol, curr_trend, streak, curr_dd)
                
                agent.update(curr_state, position_fraction, reward, next_state)
                agent.decay_epsilon()
            
            recent_profits.append(net)
            recent_returns.append(ret)
            streak = max(0, streak) + 1 if net > 0 else min(0, streak) - 1
            trades.append({'net_profit': net, 'capital': capital})
            
            if capital <= 0:
                equity_curve[i+1:] = 0
                break
        
        daily_returns = np.diff(equity_curve) / equity_curve[:-1]
        daily_returns = np.concatenate([[0], daily_returns])
        
        return equity_curve, trades, daily_returns

    def calculate_metrics(self, equity_curve, trades, daily_returns):
        """백테스팅 성과 지표 계산"""
        if len(trades) == 0:
            return self._empty_metrics()
        
        final = equity_curve[-1]
        total_ret = final - self.initial_capital
        total_ret_pct = (total_ret / self.initial_capital) * 100
        
        profits = np.array([t['net_profit'] for t in trades])
        num_trades = len(trades)
        num_wins = np.sum(profits > 0)
        win_rate = (num_wins / num_trades * 100) if num_trades > 0 else 0
        
        total_wins = np.sum(profits[profits > 0])
        total_losses = np.abs(np.sum(profits[profits < 0]))
        profit_factor = (total_wins / total_losses) if total_losses > 0 else 0
        
        running_max = np.maximum.accumulate(equity_curve)
        drawdown = (equity_curve - running_max) / running_max * 100
        max_dd = np.abs(np.min(drawdown))
        
        returns = daily_returns[daily_returns != 0]
        
        if len(returns) > 1 and np.std(returns) > 0:
            annual_ret = np.mean(returns) * 252
            annual_std = np.std(returns, ddof=1) * np.sqrt(252)
            sharpe = annual_ret / annual_std if annual_std > 0 else 0
            
            downside = returns[returns < 0]
            if len(downside) > 1:
                downside_std = np.std(downside, ddof=1) * np.sqrt(252)
                sortino = annual_ret / downside_std if downside_std > 0 else 0
            else:
                sortino = 0
        else:
            sharpe = sortino = 0
        
        years = len(equity_curve) / 252
        annual_ret_pct = (np.power(final / self.initial_capital, 1/years) - 1) * 100 if years > 0 else 0
        calmar = annual_ret_pct / max_dd if max_dd > 0 else 0
        
        if len(returns) >= self.min_trades:
            t_stat, p_value = stats.ttest_1samp(returns, 0)
            is_sig = abs(t_stat) > 2.0 and p_value < 0.05
            sufficient = True
        else:
            t_stat = p_value = np.nan
            is_sig = sufficient = False
        
        return {
            'final_capital': final,
            'total_return': total_ret,
            'total_return_pct': total_ret_pct,
            'annual_return_pct': annual_ret_pct,
            'num_trades': num_trades,
            'win_rate': win_rate,
            'profit_factor': profit_factor,
            'max_dd': max_dd,
            'sharpe': sharpe,
            'sortino': sortino,
            'calmar': calmar,
            't_stat': t_stat,
            'p_value': p_value,
            'is_sig': is_sig,
            'sufficient': sufficient
        }
    
    def _empty_metrics(self):
        return {
            'final_capital': self.initial_capital,
            'total_return': 0,
            'total_return_pct': 0,
            'annual_return_pct': 0,
            'num_trades': 0,
            'win_rate': 0,
            'profit_factor': 0,
            'max_dd': 0,
            'sharpe': 0,
            'sortino': 0,
            'calmar': 0,
            't_stat': np.nan,
            'p_value': np.nan,
            'is_sig': False,
            'sufficient': False
        }

# ========================================================================
# 5.  새로운 일관성 점수 계산 (Fold별 분석 중심)
# ========================================================================

def calculate_fold_consistency_score(fold_metrics):
    """

    
    Args:
        fold_metrics: 7개 fold의 지표 리스트
    
    Returns:
        dict: 일관성 지표들
    """
    returns = [m['total_return_pct'] for m in fold_metrics]
    sharpes = [m['sharpe'] for m in fold_metrics if not np.isnan(m['sharpe'])]
    mdds = [m['max_dd'] for m in fold_metrics]
    
    # === 1. 수익 일관성 (40점) ===
    profitable_count = sum(1 for r in returns if r > 0)
    profit_ratio = profitable_count / len(returns)
    
    # 7/7 = 40점, 6/7 = 34점, 5/7 = 28점
    consistency_1 = profit_ratio * 40
    
    # === 2. 변동성 (20점) ===
    avg_return = np.mean(returns)
    std_return = np.std(returns)
    
    if avg_return > 0:
        cv = std_return / abs(avg_return)  # Coefficient of Variation
        # CV 낮을수록 좋음: CV=0.5 → 15점, CV=1.0 → 10점, CV=2.0 → 0점
        stability = max(0, (1 - min(cv / 2, 1)) * 20)
    else:
        stability = 0
    
    # === 3. 최악 fold 성능 (20점) ===
    min_return = np.min(returns)
    max_return = np.max(returns)
    
    if max_return > 0 and min_return > 0:
        # 최악도 괜찮으면 20점
        worst_ratio = min_return / max_return
        worst_score = worst_ratio * 20
    elif min_return > 0:
        # 최악이 플러스면 15점
        worst_score = 15
    elif min_return > -5:
        # 최악이 -5% 이내면 10점
        worst_score = 10
    else:
        # 최악이 -5% 이상이면 0점
        worst_score = 0
    
    # === 4. Sharpe 일관성 (10점) ===
    avg_sharpe = np.mean(sharpes) if sharpes else 0
    if len(sharpes) > 1:
        std_sharpe = np.std(sharpes)
        # Sharpe 변동 작을수록 좋음
        sharpe_consistency = max(0, (1 - min(std_sharpe / 2, 1)) * 10)
    else:
        sharpe_consistency = 0
    
    # === 5. MDD 일관성 (10점) ===
    avg_mdd = np.mean(mdds)
    worst_mdd = np.max(mdds)
    
    # 최악 MDD 낮을수록 좋음: <10% → 10점, <15% → 7점, <20% → 5점
    if worst_mdd < 10:
        mdd_score = 10
    elif worst_mdd < 15:
        mdd_score = 7
    elif worst_mdd < 20:
        mdd_score = 5
    else:
        mdd_score = max(0, 10 - (worst_mdd - 20) / 5)
    
    # === 총점 (100점 만점) ===
    total_consistency = (
        consistency_1 +
        stability +
        worst_score +
        sharpe_consistency +
        mdd_score
    )
    
    return {
        'profitable_folds': profitable_count,
        'profit_ratio': profit_ratio,
        'avg_return_pct': avg_return,
        'std_return_pct': std_return,
        'min_return_pct': min_return,
        'max_return_pct': max_return,
        'avg_sharpe': avg_sharpe,
        'std_sharpe': np.std(sharpes) if len(sharpes) > 1 else 0,
        'avg_mdd': avg_mdd,
        'worst_mdd': worst_mdd,
        'consistency_score': total_consistency
    }

# ========================================================================
# 6. Fold별 백테스팅 (전체 15개 전략)
# ========================================================================

def run_comprehensive_backtest(fold_results_dir, output_dir):

    os.makedirs(output_dir, exist_ok=True)
    
    fold_dirs = [d for d in os.listdir(fold_results_dir) 
                 if d.startswith('fold_') and 'walk_forward' in d]
    fold_dirs = sorted(fold_dirs)
    
    print(f"\n{'='*100}")
    print(f"COMPREHENSIVE BACKTEST - ALL 15 STRATEGIES")
    print(f"{'='*100}")
    print(f"Strategies: {list(STRATEGY_INFO.keys())}")
    print(f"Folds: {len(fold_dirs)}\n")
    
    # 모델별 × 전략별 × fold별 결과 저장
    all_fold_results = {}
    
    strategies = list(STRATEGY_INFO.keys())  # 전체 15개
    thresholds = [0.1]  # 빠른 테스트
    
    # 각 Fold별로 백테스팅
    for fold_idx, fold_dir in enumerate(fold_dirs, 1):
        print(f"\n{'─'*80}")
        print(f"Fold {fold_idx}/{len(fold_dirs)}: {fold_dir}")
        print(f"{'─'*80}")
        
        fold_path = os.path.join(fold_results_dir, fold_dir)
        pred_files = [f for f in os.listdir(fold_path) if f.endswith('_predictions.csv')]
        
        for pred_file in pred_files:
            model_name = pred_file.replace('_predictions.csv', '')
            df = pd.read_csv(os.path.join(fold_path, pred_file))
            
            if not validate_dataframe(df, model_name):
                continue
            
            df['date'] = pd.to_datetime(df['date'])
            
            if model_name not in all_fold_results:
                all_fold_results[model_name] = {}
            
            # 전체 15개 전략 테스트
            for strategy in strategies:
                if strategy not in all_fold_results[model_name]:
                    all_fold_results[model_name][strategy] = {}
                
                for threshold in thresholds:
                    if threshold not in all_fold_results[model_name][strategy]:
                        all_fold_results[model_name][strategy][threshold] = []
                    
                    backtester = Backtester()
                    equity, trades, returns = backtester.backtest(df, strategy, threshold)
                    metrics = backtester.calculate_metrics(equity, trades, returns)
                    
                    metrics['fold'] = fold_idx
                    metrics['fold_name'] = fold_dir
                    metrics['model'] = model_name
                    metrics['strategy'] = strategy
                    metrics['threshold'] = threshold
                    
                    all_fold_results[model_name][strategy][threshold].append(metrics)
        
        print(f"  ✓ Processed {len(pred_files)} models × {len(strategies)} strategies")
    
    # 일관성 분석
    print(f"\n{'='*100}")
    print("FOLD CONSISTENCY ANALYSIS")
    print(f"{'='*100}\n")
    
    consistency_results = []
    
    for model_name, strategies_dict in all_fold_results.items():
        for strategy, thresholds_dict in strategies_dict.items():
            for threshold, fold_metrics in thresholds_dict.items():
                if len(fold_metrics) < 5:
                    continue
                

                consistency = calculate_fold_consistency_score(fold_metrics)
                
                consistency['model'] = model_name
                consistency['strategy'] = strategy
                consistency['threshold'] = threshold
                consistency['num_folds'] = len(fold_metrics)
                
                # 승률 평균
                win_rates = [m['win_rate'] for m in fold_metrics]
                consistency['avg_winrate'] = np.mean(win_rates)
                
                consistency_results.append(consistency)
    
    consistency_df = pd.DataFrame(consistency_results)
    

    # 1차 필터: 최소 5개 fold 수익
    filtered = consistency_df[
        (consistency_df['profitable_folds'] >= 5)
    ].copy()
    
    # 2차 정렬: 일관성 점수 기준
    top_20 = filtered.nlargest(20, 'consistency_score')
    
    # 결과 저장
    consistency_df.to_csv(os.path.join(output_dir, 'all_strategies_consistency.csv'), index=False)
    top_20.to_csv(os.path.join(output_dir, 'top20_consistent_models.csv'), index=False)
    
    # 상세 결과 저장
    detailed_results = []
    for model_name, strategies_dict in all_fold_results.items():
        for strategy, thresholds_dict in strategies_dict.items():
            for threshold, fold_metrics in thresholds_dict.items():
                detailed_results.extend(fold_metrics)
    
    detailed_df = pd.DataFrame(detailed_results)
    detailed_df.to_csv(os.path.join(output_dir, 'all_strategies_detailed.csv'), index=False)
    
    print(f"✓ Results saved:")
    print(f"  - all_strategies_consistency.csv (모든 조합)")
    print(f"  - top20_consistent_models.csv (TOP 20)")
    print(f"  - all_strategies_detailed.csv (Fold별 상세)")
    
    return top_20, consistency_df

# ========================================================================
# 7. 결과 출력
# ========================================================================

def print_top_results(top_df, num=20):
    """
    TOP 결과를 보기 좋게 출력
    """
    print(f"\n{'='*100}")
    print(f"TOP {num} CONSISTENT MODELS (Fold-wise Analysis)")
    print(f"{'='*100}\n")
    
    for rank, (idx, row) in enumerate(top_df.head(num).iterrows(), 1):
        print(f"[Rank {rank}] {row['model']} + {row['strategy']}")
        print(f"  ┌─ Consistency Score: {row['consistency_score']:.1f}/100")
        print(f"  ├─ Profitable Folds: {int(row['profitable_folds'])}/{int(row['num_folds'])} "
              f"({row['profit_ratio']*100:.0f}%)")
        print(f"  ├─ Return: Avg {row['avg_return_pct']:>6.2f}% (±{row['std_return_pct']:.2f}%), "
              f"Range [{row['min_return_pct']:>6.2f}% ~ {row['max_return_pct']:>6.2f}%]")
        print(f"  ├─ Sharpe: Avg {row['avg_sharpe']:>5.2f} (±{row['std_sharpe']:.2f})")
        print(f"  ├─ MDD: Avg {row['avg_mdd']:>5.1f}%, Worst {row['worst_mdd']:>5.1f}%")
        print(f"  └─ Win Rate: {row['avg_winrate']:.1f}%")
        print()

# ========================================================================
# 8. 메인 실행
# ========================================================================

def main():
    """전체 15개 전략으로 백테스팅 실행"""
    print("="*100)
    print("ETHEREUM TRADING BACKTEST - ALL 15 STRATEGIES")
    print("="*100)
    print(f"Input: {fold_results_dir}")
    print(f"Output: {output_dir}")
    
    # 전체 전략 백테스팅
    top_20, full_results = run_comprehensive_backtest(fold_results_dir, output_dir)
    
    # 결과 출력
    print_top_results(top_20, num=20)
    
    # 전략별 Best 모델
    print(f"\n{'='*100}")
    print("BEST MODEL PER STRATEGY")
    print(f"{'='*100}\n")
    
    for strategy in STRATEGY_INFO.keys():
        strategy_best = full_results[
            full_results['strategy'] == strategy
        ].nlargest(1, 'consistency_score')
        
        if len(strategy_best) > 0:
            row = strategy_best.iloc[0]
            print(f"{strategy.upper()}: {row['model']} "
                  f"(Score: {row['consistency_score']:.1f}, "
                  f"Folds: {int(row['profitable_folds'])}/7, "
                  f"Avg Return: {row['avg_return_pct']:.2f}%)")
    
    print(f"\n{'='*100}")
    print("BACKTEST COMPLETED!")
    print(f"{'='*100}")

if __name__ == "__main__":
    main()


ETHEREUM TRADING BACKTEST - ALL 15 STRATEGIES
Input: ../model_results/2025-10-26/fold_results/direction
Output: ../model_results/2025-10-26/backtest_results

COMPREHENSIVE BACKTEST - ALL 15 STRATEGIES
Strategies: ['kelly', 'aggressive', 'neutral', 'conservative', 'ultra_aggressive', 'fixed_50', 'fixed_01', 'volatility_scaled', 'inverse_volatility', 'trend_following', 'momentum', 'reinforcement_learning', 'regime_adaptive_balanced', 'regime_adaptive_defensive', 'regime_adaptive_aggressive']
Folds: 7


────────────────────────────────────────────────────────────────────────────────
Fold 1/7: fold_1_walk_forward_rolling_reverse
────────────────────────────────────────────────────────────────────────────────
  ✓ Processed 29 models × 15 strategies

────────────────────────────────────────────────────────────────────────────────
Fold 2/7: fold_2_walk_forward_rolling_reverse
────────────────────────────────────────────────────────────────────────────────
  ✓ Processed 29 models × 15 strategi

In [7]:
import pandas as pd
import numpy as np
import os
from scipy import stats
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

timestamp = '2025-10-26'
RESULT_DIR = os.path.join("../model_results", timestamp)
fold_results_dir = os.path.join(RESULT_DIR, "fold_results", "direction")
raw_data_dir = os.path.join(RESULT_DIR, "raw_data", "direction", "walk_forward")
output_dir = os.path.join(RESULT_DIR, "backtest_results_spot")

STRATEGY_INFO = {
    'kelly': 'Kelly Criterion', 
    'aggressive': 'Fixed 30%', 
    'neutral': 'Confidence 15%',
    'conservative': 'Fixed 5%', 
    'ultra_aggressive': 'Fixed 80%', 
    'fixed_50': 'Fixed 50%',
    'fixed_01': 'Fixed 1%', 
    'volatility_scaled': 'Inverse Vol cap 30%',
    'inverse_volatility': 'Inverse Vol cap 50%', 
    'trend_following': 'Momentum Binary',
    'momentum': 'Momentum Scaled',
    'regime_adaptive_balanced': 'Regime Balanced', 
    'regime_adaptive_defensive': 'Regime Defensive',
    'regime_adaptive_aggressive': 'Regime Aggressive'
}

def validate_dataframe(df, model_name):
    required_cols = ['date', 'pred_direction', 'actual_return', 'correct', 'confidence', 'max_proba']
    missing = [col for col in required_cols if col not in df.columns]
    
    if missing or len(df) < 10 or df['actual_return'].isna().all():
        return False
    return True

def calculate_market_benchmark(test_raw_path):
    if not os.path.exists(test_raw_path):
        return None
    
    df = pd.read_csv(test_raw_path)
    
    if 'next_open' not in df.columns or 'next_close' not in df.columns:
        return None
    
    cumulative_return = 1.0
    daily_returns = []
    
    for i in range(len(df)):
        open_price = df.iloc[i]['next_open']
        close_price = df.iloc[i]['next_close']
        
        daily_ret = (close_price - open_price) / open_price
        cumulative_return *= (1 + daily_ret)
        daily_returns.append(daily_ret)
    
    total_return_pct = (cumulative_return - 1) * 100
    
    if len(df) > 5:
        first_open = df.iloc[0]['next_open']
        last_close = df.iloc[-1]['next_close']
        period_return = (last_close - first_open) / first_open
        
        if period_return > 0.05:
            regime = "BULL"
        elif period_return < -0.05:
            regime = "BEAR"
        else:
            regime = "SIDEWAYS"
    else:
        regime = "UNKNOWN"
    
    return {
        'total_return_pct': total_return_pct,
        'daily_returns': daily_returns,
        'regime': regime,
        'num_days': len(df)
    }

class SpotBacktester:
    def __init__(self, initial_capital=1000000000, fee=0.0005, slippage=0.0015, min_order=5000):
        self.initial_capital = float(initial_capital)
        self.fee = float(fee)
        self.slippage = float(slippage)
        self.min_order = min_order
        self.min_trades = 30

    def detect_regime(self, returns, lookback=30):
        if len(returns) < lookback:
            return "sideways", 0.15
        
        recent = returns[-lookback:]
        cum_ret = np.sum(recent)
        volatility = np.std(recent)
        
        if cum_ret > 0.10 and volatility > 0.025:
            return "bull_strong", 0.40
        elif cum_ret > 0.05 and volatility < 0.020:
            return "bull_steady", 0.30
        elif cum_ret < -0.10 and volatility > 0.025:
            return "bear_crash", 0.05
        elif cum_ret < -0.05:
            return "bear_volatile", 0.08
        else:
            return "sideways", 0.15

    def kelly_fraction(self, profits):
        if len(profits) < 10:
            return 0.05
        
        profits = np.array(profits)
        wins = profits[profits > 0]
        losses = profits[profits < 0]
        
        if len(wins) == 0 or len(losses) == 0 or np.mean(losses) == 0:
            return 0.05
        
        win_rate = len(wins) / len(profits)
        avg_win = np.mean(wins)
        avg_loss = np.abs(np.mean(losses))
        win_loss_ratio = avg_win / avg_loss
        kelly = (win_rate * win_loss_ratio - (1 - win_rate)) / win_loss_ratio
        
        return np.clip(kelly * 0.5, 0.01, 0.2)

    def calculate_position(self, df, i, strategy, recent_profits, recent_returns):
        confidence = df.iloc[i]['confidence']
        max_proba = df.iloc[i]['max_proba']
        
        if strategy == 'kelly':
            if len(recent_profits) >= 20:
                return self.kelly_fraction(recent_profits[-50:])
            return 0.05
        
        elif strategy == 'aggressive':
            return 0.3
        
        elif strategy == 'neutral':
            return np.clip(max_proba * 0.15, 0.01, 0.15)
        
        elif strategy == 'conservative':
            return 0.05
        
        elif strategy == 'ultra_aggressive':
            return 0.8
        
        elif strategy == 'fixed_50':
            return 0.5
        
        elif strategy == 'fixed_01':
            return 0.01
        
        elif strategy == 'volatility_scaled':
            if len(recent_returns) >= 20:
                vol = np.std(recent_returns[-20:])
                vol = max(vol, 0.01)
                return np.clip(0.1 / vol, 0.01, 0.3)
            return 0.1
        
        elif strategy == 'inverse_volatility':
            if len(recent_returns) >= 20:
                vol = np.std(recent_returns[-20:])
                vol = max(vol, 0.01)
                return np.clip(0.2 / vol, 0.01, 0.5)
            return 0.15
        
        elif strategy == 'trend_following':
            if len(recent_returns) >= 10:
                trend = np.sum(recent_returns[-10:])
                return 0.3 if trend > 0 else 0.05
            return 0.1
        
        elif strategy == 'momentum':
            if len(recent_returns) >= 10:
                momentum = np.sum(recent_returns[-10:])
                return np.clip(0.1 + 0.2 * (momentum / 0.1), 0.01, 0.3)
            return 0.1
        
        elif strategy == 'regime_adaptive_balanced':
            regime, base_pos = self.detect_regime(recent_returns, 30)
            multipliers = {
                "bear_crash": 0.8, "bear_volatile": 0.9, "sideways": 1.0, 
                "bull_steady": 1.3, "bull_strong": 1.5
            }
            mult = multipliers.get(regime, 1.0)
            conf_mult = 0.8 + (confidence * 0.8)
            final_pos = base_pos * mult * conf_mult
            
            if regime in ["bear_crash", "bear_volatile"]:
                return np.clip(final_pos, 0.01, 0.15)
            else:
                return np.clip(final_pos, 0.01, 0.7)
        
        elif strategy == 'regime_adaptive_defensive':
            regime, base_pos = self.detect_regime(recent_returns, 30)
            multipliers = {
                "bear_crash": 0.6, "bear_volatile": 0.7, "sideways": 0.8, 
                "bull_steady": 1.0, "bull_strong": 1.1
            }
            mult = multipliers.get(regime, 1.0)
            conf_mult = 0.8 + (confidence * 0.8)
            final_pos = base_pos * mult * conf_mult
            
            if regime in ["bear_crash", "bear_volatile"]:
                return np.clip(final_pos, 0.01, 0.15)
            else:
                return np.clip(final_pos, 0.01, 0.7)
        
        elif strategy == 'regime_adaptive_aggressive':
            regime, base_pos = self.detect_regime(recent_returns, 30)
            multipliers = {
                "bear_crash": 1.0, "bear_volatile": 1.2, "sideways": 1.5, 
                "bull_steady": 1.8, "bull_strong": 2.0
            }
            mult = multipliers.get(regime, 1.0)
            conf_mult = 0.8 + (confidence * 0.8)
            final_pos = base_pos * mult * conf_mult
            
            if regime in ["bear_crash", "bear_volatile"]:
                return np.clip(final_pos, 0.01, 0.15)
            else:
                return np.clip(final_pos, 0.01, 0.7)
        
        return 0.1

    def backtest(self, df, strategy, confidence_threshold=0.1):
        df = df.copy().sort_values('date').reset_index(drop=True)
        n = len(df)
        
        capital = self.initial_capital
        equity_curve = np.zeros(n)
        trades = []
        recent_profits = []
        recent_returns = []
        peak = capital
        
        for i in range(n):
            direction = int(df.iloc[i]['pred_direction'])
            ret = float(df.iloc[i]['actual_return'])
            
            if direction == 0:
                equity_curve[i] = capital
                continue
            
            if capital < self.min_order or df.iloc[i]['confidence'] < confidence_threshold:
                equity_curve[i] = capital
                continue
            
            position_fraction = self.calculate_position(
                df, i, strategy, recent_profits, recent_returns
            )
            position_size = capital * position_fraction
            
            if position_size < self.min_order:
                equity_curve[i] = capital
                continue
            
            volatility = np.std(recent_returns[-20:]) if len(recent_returns) >= 20 else 0.02
            dynamic_slippage = self.slippage * (2 if volatility > 0.03 else 1.5 if volatility > 0.02 else 1.0)
            cost = position_size * ((self.fee + dynamic_slippage) * 2)
            
            gross = position_size * ret
            net = gross - cost
            
            capital = max(0, capital + net)
            peak = max(peak, capital)
            equity_curve[i] = capital
            
            recent_profits.append(net)
            recent_returns.append(ret)
            trades.append({
                'date': df.iloc[i]['date'],
                'net_profit': net,
                'capital': capital,
                'position_fraction': position_fraction
            })
            
            if capital <= 0:
                equity_curve[i+1:] = 0
                break
        
        daily_returns = np.diff(equity_curve) / equity_curve[:-1]
        daily_returns = np.concatenate([[0], daily_returns])
        
        return equity_curve, trades, daily_returns

    def calculate_metrics(self, equity_curve, trades, daily_returns):
        if len(trades) == 0:
            return self._empty_metrics()
        
        final = equity_curve[-1]
        total_ret = final - self.initial_capital
        total_ret_pct = (total_ret / self.initial_capital) * 100
        
        profits = np.array([t['net_profit'] for t in trades])
        num_trades = len(trades)
        num_wins = np.sum(profits > 0)
        win_rate = (num_wins / num_trades * 100) if num_trades > 0 else 0
        
        total_wins = np.sum(profits[profits > 0])
        total_losses = np.abs(np.sum(profits[profits < 0]))
        profit_factor = (total_wins / total_losses) if total_losses > 0 else 0
        
        running_max = np.maximum.accumulate(equity_curve)
        drawdown = (equity_curve - running_max) / running_max * 100
        max_dd = np.abs(np.min(drawdown))
        
        returns = daily_returns[daily_returns != 0]
        
        if len(returns) > 1 and np.std(returns) > 0:
            annual_ret = np.mean(returns) * 252
            annual_std = np.std(returns, ddof=1) * np.sqrt(252)
            sharpe = annual_ret / annual_std if annual_std > 0 else 0
            
            downside = returns[returns < 0]
            if len(downside) > 1:
                downside_std = np.std(downside, ddof=1) * np.sqrt(252)
                sortino = annual_ret / downside_std if downside_std > 0 else 0
            else:
                sortino = 0
        else:
            sharpe = sortino = 0
        
        years = len(equity_curve) / 252
        annual_ret_pct = (np.power(final / self.initial_capital, 1/years) - 1) * 100 if years > 0 else 0
        calmar = annual_ret_pct / max_dd if max_dd > 0 else 0
        
        if len(returns) >= self.min_trades:
            t_stat, p_value = stats.ttest_1samp(returns, 0)
            is_sig = abs(t_stat) > 2.0 and p_value < 0.05
            sufficient = True
        else:
            t_stat = p_value = np.nan
            is_sig = sufficient = False
        
        return {
            'final_capital': final,
            'total_return': total_ret,
            'total_return_pct': total_ret_pct,
            'annual_return_pct': annual_ret_pct,
            'num_trades': num_trades,
            'win_rate': win_rate,
            'profit_factor': profit_factor,
            'max_dd': max_dd,
            'sharpe': sharpe,
            'sortino': sortino,
            'calmar': calmar,
            't_stat': t_stat,
            'p_value': p_value,
            'is_sig': is_sig,
            'sufficient': sufficient
        }
    
    def _empty_metrics(self):
        return {
            'final_capital': self.initial_capital,
            'total_return': 0,
            'total_return_pct': 0,
            'annual_return_pct': 0,
            'num_trades': 0,
            'win_rate': 0,
            'profit_factor': 0,
            'max_dd': 0,
            'sharpe': 0,
            'sortino': 0,
            'calmar': 0,
            't_stat': np.nan,
            'p_value': np.nan,
            'is_sig': False,
            'sufficient': False
        }

def calculate_fold_consistency_score(fold_metrics):
    returns = [m['total_return_pct'] for m in fold_metrics]
    sharpes = [m['sharpe'] for m in fold_metrics if not np.isnan(m['sharpe'])]
    mdds = [m['max_dd'] for m in fold_metrics]
    
    profitable_count = sum(1 for r in returns if r > 0)
    profit_ratio = profitable_count / len(returns)
    consistency_1 = profit_ratio * 40
    
    avg_return = np.mean(returns)
    std_return = np.std(returns)
    
    if avg_return > 0:
        cv = std_return / abs(avg_return)
        stability = max(0, (1 - min(cv / 2, 1)) * 20)
    else:
        stability = 0
    
    min_return = np.min(returns)
    max_return = np.max(returns)
    
    if max_return > 0 and min_return > 0:
        worst_ratio = min_return / max_return
        worst_score = worst_ratio * 20
    elif min_return > 0:
        worst_score = 15
    elif min_return > -5:
        worst_score = 10
    else:
        worst_score = 0
    
    avg_sharpe = np.mean(sharpes) if sharpes else 0
    if len(sharpes) > 1:
        std_sharpe = np.std(sharpes)
        sharpe_consistency = max(0, (1 - min(std_sharpe / 2, 1)) * 10)
    else:
        sharpe_consistency = 0
    
    avg_mdd = np.mean(mdds)
    worst_mdd = np.max(mdds)
    
    if worst_mdd < 10:
        mdd_score = 10
    elif worst_mdd < 15:
        mdd_score = 7
    elif worst_mdd < 20:
        mdd_score = 5
    else:
        mdd_score = max(0, 10 - (worst_mdd - 20) / 5)
    
    total_consistency = consistency_1 + stability + worst_score + sharpe_consistency + mdd_score
    
    return {
        'profitable_folds': profitable_count,
        'profit_ratio': profit_ratio,
        'avg_return_pct': avg_return,
        'std_return_pct': std_return,
        'min_return_pct': min_return,
        'max_return_pct': max_return,
        'avg_sharpe': avg_sharpe,
        'std_sharpe': np.std(sharpes) if len(sharpes) > 1 else 0,
        'avg_mdd': avg_mdd,
        'worst_mdd': worst_mdd,
        'consistency_score': total_consistency
    }

def run_comprehensive_backtest(fold_results_dir, raw_data_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    
    fold_dirs = [d for d in os.listdir(fold_results_dir) 
                 if d.startswith('fold_') and 'walk_forward' in d]
    fold_dirs = sorted(fold_dirs)
    
    print(f"\n{'='*100}")
    print(f"SPOT TRADING BACKTEST WITH ALPHA ANALYSIS")
    print(f"{'='*100}")
    print(f"Trading Mode: SPOT (Long Only)")
    print(f"Strategies: {len(STRATEGY_INFO)}")
    print(f"Folds: {len(fold_dirs)}\n")
    
    all_fold_results = {}
    market_benchmarks = []
    
    strategies = list(STRATEGY_INFO.keys())
    thresholds = [0.1]
    
    for fold_idx, fold_dir in enumerate(fold_dirs, 1):
        print(f"\nFold {fold_idx}/{len(fold_dirs)}: {fold_dir}")
        
        test_raw_path = os.path.join(raw_data_dir, fold_dir, 'test_raw.csv')
        market_bench = calculate_market_benchmark(test_raw_path)
        
        if market_bench:
            market_bench['fold'] = fold_idx
            market_bench['fold_name'] = fold_dir
            market_benchmarks.append(market_bench)
            print(f"  Market: {market_bench['regime']} ({market_bench['total_return_pct']:+.2f}%)")
        
        fold_path = os.path.join(fold_results_dir, fold_dir)
        pred_files = [f for f in os.listdir(fold_path) if f.endswith('_predictions.csv')]
        
        for pred_file in pred_files:
            model_name = pred_file.replace('_predictions.csv', '')
            df = pd.read_csv(os.path.join(fold_path, pred_file))
            
            if not validate_dataframe(df, model_name):
                continue
            
            df['date'] = pd.to_datetime(df['date'])
            
            if model_name not in all_fold_results:
                all_fold_results[model_name] = {}
            
            for strategy in strategies:
                if strategy not in all_fold_results[model_name]:
                    all_fold_results[model_name][strategy] = {}
                
                for threshold in thresholds:
                    if threshold not in all_fold_results[model_name][strategy]:
                        all_fold_results[model_name][strategy][threshold] = []
                    
                    backtester = SpotBacktester()
                    equity, trades, returns = backtester.backtest(df, strategy, threshold)
                    metrics = backtester.calculate_metrics(equity, trades, returns)
                    
                    metrics['fold'] = fold_idx
                    metrics['fold_name'] = fold_dir
                    metrics['model'] = model_name
                    metrics['strategy'] = strategy
                    metrics['threshold'] = threshold
                    
                    if market_bench:
                        metrics['market_return_pct'] = market_bench['total_return_pct']
                        metrics['market_regime'] = market_bench['regime']
                        metrics['alpha'] = metrics['total_return_pct'] - market_bench['total_return_pct']
                    else:
                        metrics['market_return_pct'] = 0
                        metrics['market_regime'] = 'UNKNOWN'
                        metrics['alpha'] = 0
                    
                    all_fold_results[model_name][strategy][threshold].append(metrics)
        
        print(f"  Processed {len(pred_files)} models")
    
    market_bench_df = pd.DataFrame(market_benchmarks)
    market_bench_df.to_csv(os.path.join(output_dir, 'market_benchmarks.csv'), index=False)
    
    print(f"\n{'='*100}")
    print("FOLD CONSISTENCY AND ALPHA ANALYSIS")
    print(f"{'='*100}\n")
    
    consistency_results = []
    
    for model_name, strategies_dict in all_fold_results.items():
        for strategy, thresholds_dict in strategies_dict.items():
            for threshold, fold_metrics in thresholds_dict.items():
                if len(fold_metrics) < 5:
                    continue
                
                consistency = calculate_fold_consistency_score(fold_metrics)
                
                consistency['model'] = model_name
                consistency['strategy'] = strategy
                consistency['threshold'] = threshold
                consistency['num_folds'] = len(fold_metrics)
                
                win_rates = [m['win_rate'] for m in fold_metrics]
                consistency['avg_winrate'] = np.mean(win_rates)
                
                alphas = [m['alpha'] for m in fold_metrics]
                consistency['avg_alpha'] = np.mean(alphas)
                consistency['positive_alpha_folds'] = sum(1 for a in alphas if a > 0)
                
                bull_metrics = [m for m in fold_metrics if m['market_regime'] == 'BULL']
                bear_metrics = [m for m in fold_metrics if m['market_regime'] == 'BEAR']
                
                if bull_metrics:
                    bull_alphas = [m['alpha'] for m in bull_metrics]
                    consistency['bull_alpha'] = np.mean(bull_alphas)
                else:
                    consistency['bull_alpha'] = 0
                
                if bear_metrics:
                    bear_alphas = [m['alpha'] for m in bear_metrics]
                    consistency['bear_alpha'] = np.mean(bear_alphas)
                else:
                    consistency['bear_alpha'] = 0
                
                consistency_results.append(consistency)
    
    consistency_df = pd.DataFrame(consistency_results)
    
    filtered = consistency_df[
        (consistency_df['profitable_folds'] >= 5)
    ].copy()
    
    top_20 = filtered.nlargest(20, 'avg_alpha')
    
    consistency_df.to_csv(os.path.join(output_dir, 'all_strategies_consistency.csv'), index=False)
    top_20.to_csv(os.path.join(output_dir, 'top20_consistent_models.csv'), index=False)
    
    detailed_results = []
    for model_name, strategies_dict in all_fold_results.items():
        for strategy, thresholds_dict in strategies_dict.items():
            for threshold, fold_metrics in thresholds_dict.items():
                detailed_results.extend(fold_metrics)
    
    detailed_df = pd.DataFrame(detailed_results)
    detailed_df.to_csv(os.path.join(output_dir, 'all_strategies_detailed.csv'), index=False)
    
    print(f"\nResults saved:")
    print(f"  - market_benchmarks.csv")
    print(f"  - all_strategies_consistency.csv")
    print(f"  - top20_consistent_models.csv")
    print(f"  - all_strategies_detailed.csv")
    
    return top_20, consistency_df, market_bench_df

def print_top_results(top_df, num=20):
    print(f"\n{'='*100}")
    print(f"TOP {num} MODELS (SPOT TRADING - LONG ONLY)")
    print(f"{'='*100}\n")
    
    for rank, (idx, row) in enumerate(top_df.head(num).iterrows(), 1):
        print(f"[Rank {rank}] {row['model']} + {row['strategy']}")
        print(f"  Consistency Score: {row['consistency_score']:.1f}/100")
        print(f"  Profitable Folds: {int(row['profitable_folds'])}/{int(row['num_folds'])} "
              f"({row['profit_ratio']*100:.0f}%)")
        print(f"  Model Return: Avg {row['avg_return_pct']:>6.2f}% (±{row['std_return_pct']:.2f}%)")
        print(f"  Alpha: Avg {row['avg_alpha']:>+6.2f}% ({int(row['positive_alpha_folds'])}/7 positive)")
        print(f"  Bull Alpha: {row['bull_alpha']:>+6.2f}%")
        print(f"  Bear Alpha: {row['bear_alpha']:>+6.2f}%")
        print(f"  Sharpe: Avg {row['avg_sharpe']:>5.2f}")
        print(f"  MDD: Worst {row['worst_mdd']:>5.1f}%")
        print()

def main():
    print("="*100)
    print("ETHEREUM SPOT TRADING BACKTEST")
    print("="*100)
    print(f"Input: {fold_results_dir}")
    print(f"Output: {output_dir}")
    
    top_20, full_results, market_benchmarks = run_comprehensive_backtest(
        fold_results_dir, raw_data_dir, output_dir
    )
    
    print_top_results(top_20, num=20)
    
    print(f"\n{'='*100}")
    print("MARKET REGIME SUMMARY")
    print(f"{'='*100}\n")
    
    regime_counts = market_benchmarks['regime'].value_counts()
    for regime, count in regime_counts.items():
        print(f"{regime}: {count} folds")
    
    print(f"\n{'='*100}")
    print("BACKTEST COMPLETED")
    print(f"{'='*100}")

if __name__ == "__main__":
    main()


ETHEREUM SPOT TRADING BACKTEST
Input: ../model_results/2025-10-26/fold_results/direction
Output: ../model_results/2025-10-26/backtest_results_spot

SPOT TRADING BACKTEST WITH ALPHA ANALYSIS
Trading Mode: SPOT (Long Only)
Strategies: 14
Folds: 7


Fold 1/7: fold_1_walk_forward_rolling_reverse
  Market: BULL (+52.52%)
  Processed 29 models

Fold 2/7: fold_2_walk_forward_rolling_reverse
  Market: BEAR (-12.25%)
  Processed 29 models

Fold 3/7: fold_3_walk_forward_rolling_reverse
  Market: BULL (+105.18%)
  Processed 29 models

Fold 4/7: fold_4_walk_forward_rolling_reverse
  Market: SIDEWAYS (-3.40%)
  Processed 29 models

Fold 5/7: fold_5_walk_forward_rolling_reverse
  Market: SIDEWAYS (+0.60%)
  Processed 29 models

Fold 6/7: fold_6_walk_forward_rolling_reverse
  Market: BEAR (-23.45%)
  Processed 29 models

Fold 7/7: fold_7_walk_forward_rolling_reverse
  Market: BULL (+49.49%)
  Processed 29 models

FOLD CONSISTENCY AND ALPHA ANALYSIS


Results saved:
  - market_benchmarks.csv
  - all_s